<a href="https://colab.research.google.com/github/ColinJ69/HydroHaven/blob/main/predict_pfas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

df = pd.read_excel('pfas (1).xlsx')
t1 = []
num = 0
labels = []
sources = []

#print(df.loc[156].tolist()[2:])

features = ['Well depth', 'PP_1km', 'PP_4km', 'PP_12km','AFFF', 'KP_50km', 'Nearest_KP', 'AFF_Acc', 'Basin', 'KP_Name', 'AFFF_Con', 'pm10', 'pm25', 'so', 'lat', 'lng', 'co', 'no','silt', 'slope', 'water','temp', 'humid', 'dew', 'rain', 'evapo', 'wind']
for i in range(0, 340):


    #print(df.loc[i]['Split'])
    labels.append(int(df.loc[i]['Split']))
    sources.append(df.loc[i]['County'])

    #t1.append(df.loc[i].tolist()[2:])
    numeric_row = pd.to_numeric(df.loc[i].tolist()[2:34], errors='coerce')

    t1.append(numeric_row.tolist())

print(df['Split'].value_counts())
print(t1[0])

Split
0    268
1     72
Name: count, dtype: int64
[36.044422, -79.536914, 720.0, 80042.0, 4.0, 5.0, 18.0, 5.0, 10.65, 37.0, 1.0, 21.85733397271898, 88.40057299931844, 19.69386168585883, 0.4969444446472658, 0.1062786542539672, 9.569445922142929, 21.78431311978234, 0.4322277778345677, 13.13, 11.57, 0.19, 220.24, 2.23, 35.47, 0.0, 30.03, 8.05, 0.137, 6.0, 2.0, 3.0]


In [23]:
from sklearn .metrics import roc_auc_score, f1_score, recall_score, precision_score

In [ ]:
print(sum(labels)/130)

0.5538461538461539


In [24]:
from imblearn.over_sampling import ADASYN

In [25]:
adasyn = ADASYN(sampling_strategy='minority')


In [26]:
from imblearn.over_sampling import SMOTE

In [27]:
sm = SMOTE(random_state=12)
sm.fit(t1, labels)
#t1, labels = sm.fit_resample(t1, labels)
sm.fit(t1, labels)
t1, labels = sm.fit_resample(t1, labels)


In [ ]:
print(df['Split'].isnull().sum())
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='not minority', random_state=12)
df_balanced, balanced_labels = rus.fit_resample(t1, labels)
print(pd.Series(balanced_labels).value_counts())

0
0    72
1    72
Name: count, dtype: int64


In [ ]:
df_balanced = df_balanced.drop('County', axis=1)
df_balanced = df_balanced.drop('Split', axis=1)
df_balanced = np.array(df_balanced)
print(round(len(sorted(balanced_labels)[sorted(balanced_labels).index(1):])/len(balanced_labels),2))

AttributeError: 'list' object has no attribute 'drop'

In [36]:

X_train, X_test, y_train, y_test = train_test_split(t1, labels, test_size=0.2, random_state=12)


print(pd.Series(y_test).value_counts())


rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
auc = np.round(roc_auc_score(y_test, y_pred), 10)
print(auc)
f1 = np.round(f1_score(y_test, y_pred), 10)
print(f1)
recall = np.round(recall_score(y_test, y_pred), 10)
print(recall)
precision = np.round(precision_score(y_test, y_pred, zero_division=1), 10)
print(precision)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


#print(np.max(rf.predict_proba(X_testt), axis=1))

0    57
1    51
Name: count, dtype: int64
0.9287925697
0.9259259259
0.9803921569
0.8771929825
Accuracy: 92.59%


In [29]:

importances = rf.feature_importances_
std = list(np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0))
e = dict((n, l) for n, l in zip(features, std))
print(len(std))
sorted_dict = {k: round((v *100), 2)  for k, v in reversed(sorted(e.items(), key=lambda item: item[1]))}
print(sorted_dict)

32
{'Nearest_KP': 14.8, 'temp': 13.83, 'evapo': 10.63, 'KP_50km': 6.85, 'AFFF_Con': 6.54, 'pm25': 6.51, 'AFF_Acc': 6.16, 'pm10': 5.17, 'no': 5.01, 'so': 4.8, 'PP_1km': 4.57, 'AFFF': 4.37, 'water': 3.04, 'co': 2.85, 'lng': 2.44, 'lat': 2.37, 'KP_Name': 2.13, 'slope': 2.1, 'Well depth': 1.96, 'Basin': 1.91, 'rain': 1.91, 'humid': 1.79, 'dew': 1.76, 'PP_12km': 1.7, 'PP_4km': 1.69, 'wind': 1.5, 'silt': 1.32}


In [37]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(t1, labels, test_size=0.2, random_state=42)
model = xgb.XGBClassifier(n_estimators=100, eval_metric='logloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)

print(f'Accuracy: {accuracy * 100:.2f}%')
auc = np.round(roc_auc_score(y_test, y_pred), 10)
print(auc)
f1 = np.round(f1_score(y_test, y_pred), 10)
print(f1)
recall = np.round(recall_score(y_test, y_pred), 10)
print(recall)
precision = np.round(precision_score(y_test, y_pred), 10)
print(precision)


Accuracy: 93.52%
0.9354395604
0.9333333333
0.9423076923
0.9245283019


In [38]:
from sklearn.naive_bayes import GaussianNB

# Example usage
model = GaussianNB()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')
auc = np.round(roc_auc_score(y_test, y_pred), 10)
print(auc)
f1 = np.round(f1_score(y_test, y_pred), 10)
print(f1)
recall = np.round(recall_score(y_test, y_pred), 10)
print(recall)
precision = np.round(precision_score(y_test, y_pred), 10)
print(precision)

Accuracy: 85.19%
0.9354395604
0.9333333333
0.9423076923
0.9245283019


In [39]:
from sklearn.linear_model import LogisticRegression

modellgr = LogisticRegression(max_iter=100)
modellgr.fit(X_train, y_train)
y_predlgr = modellgr.predict(X_test)
accuracylgr = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracylgr}')
auclgr = np.round(roc_auc_score(y_test, y_pred), 10)
print(auclgr)
f1lgr = np.round(f1_score(y_test, y_pred), 10)
print(f1lgr)
recalllgr = np.round(recall_score(y_test, y_pred), 10)
print(recalllgr)
precisionlgr = np.round(precision_score(y_test, y_pred), 10)
print(precisionlgr)



Accuracy: 0.9351851851851852
0.9354395604
0.9333333333
0.9423076923
0.9245283019


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
!pip3 install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.0 MB/s eta 0:00:00


In [40]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(loss_function='Logloss')
model.fit(X_train, y_train, verbose=100)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
auc = np.round(roc_auc_score(y_test, y_pred), 10)
print(auc)
f1 = np.round(f1_score(y_test, y_pred), 10)
print(f1)
recall = np.round(recall_score(y_test, y_pred), 10)
print(recall)
precision = np.round(precision_score(y_test, y_pred), 10)
print(precision)

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

Learning rate set to 0.007171
0:	learn: 0.6857792	total: 7.66ms	remaining: 7.66s
100:	learn: 0.3310009	total: 718ms	remaining: 6.39s
200:	learn: 0.2185737	total: 1.42s	remaining: 5.65s
300:	learn: 0.1640088	total: 2.13s	remaining: 4.96s
400:	learn: 0.1304580	total: 2.83s	remaining: 4.23s
500:	learn: 0.1082823	total: 3.52s	remaining: 3.51s
600:	learn: 0.0913146	total: 4.26s	remaining: 2.83s
700:	learn: 0.0777673	total: 4.96s	remaining: 2.11s
800:	learn: 0.0672489	total: 5.65s	remaining: 1.4s
900:	learn: 0.0579767	total: 6.36s	remaining: 699ms
999:	learn: 0.0508863	total: 7.06s	remaining: 0us
Accuracy: 0.9629629629629629
0.9629120879
0.9615384615
0.9615384615
0.9615384615


In [ ]:
probabilities = model.predict_proba(X_test)
print(X_test[3])
for i, prob in enumerate(probabilities):
    print(f"Input {i}: Class 0 confidence = {prob[0]:.4f}, Class 1 confidence = {prob[1]:.4f}")

[0.0, 1.0, 4.0, 31.0, 2.0, 3.0, 21.73, 1.0, 1.0, 27.0, 3.0, 13.93, 11.62, 0.11, 35.862601, -79.828221, 179.07, 3.02, 32.18, 8.89, 0.113, 21.78275065951877, 87.97759965260823, 19.51344503296746, 0.4741666698414418, 0.1070007027293387, 9.71981692140301]
Input 0: Class 0 confidence = 0.9855, Class 1 confidence = 0.0145
Input 1: Class 0 confidence = 0.4633, Class 1 confidence = 0.5367
Input 2: Class 0 confidence = 0.7452, Class 1 confidence = 0.2548
Input 3: Class 0 confidence = 0.0155, Class 1 confidence = 0.9845
Input 4: Class 0 confidence = 0.9704, Class 1 confidence = 0.0296
Input 5: Class 0 confidence = 0.9722, Class 1 confidence = 0.0278
Input 6: Class 0 confidence = 0.9876, Class 1 confidence = 0.0124
Input 7: Class 0 confidence = 0.0978, Class 1 confidence = 0.9022
Input 8: Class 0 confidence = 0.9428, Class 1 confidence = 0.0572
Input 9: Class 0 confidence = 0.4195, Class 1 confidence = 0.5805
Input 10: Class 0 confidence = 0.0367, Class 1 confidence = 0.9633
Input 11: Class 0 con

In [ ]:
!pip install dask-expr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.6/242.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.1 MB/s eta 0:00:00
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.1
    Uninstalling cloudpickle-2.2.1:
      Successfully uninstalled cloudpickle-2.2.1
  Attempting uninstall: dask
    Found existing installation: dask 2024.8.0
    Uninstalling dask-2024.8.0:
      Successfully uninstalled dask-2024.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2024.8.0 requires dask==2024.8.0, but you have dask 2024.9.0 which is incompatible.


In [41]:
import lightgbm as lgb

model = lgb.LGBMClassifier(force_col_wise=True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
auc = np.round(roc_auc_score(y_test, y_pred), 10)
print(auc)
f1 = np.round(f1_score(y_test, y_pred), 10)
print(f1)
recall = np.round(recall_score(y_test, y_pred), 10)
print(recall)
precision = np.round(precision_score(y_test, y_pred), 10)
print(precision)

[LightGBM] [Info] Number of positive: 216, number of negative: 212
[LightGBM] [Info] Total Bins 3360
[LightGBM] [Info] Number of data points in the train set: 428, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504673 -> initscore=0.018692
[LightGBM] [Info] Start training from score 0.018692
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [42]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
auc = np.round(roc_auc_score(y_test, y_pred), 10)
print(auc)
f1 = np.round(f1_score(y_test, y_pred), 10)
print(f1)
recall = np.round(recall_score(y_test, y_pred), 10)
print(recall)
precision = np.round(precision_score(y_test, y_pred), 10)
print(precision)


Accuracy: 0.7685185185185185
0.7651098901
0.7368421053
0.6730769231
0.8139534884
